# Gemma3 through vLLM on Sagemaker

## Gemma3 : SLM(Small Language Model) 
[특징]
- 다양한 모델 크기 : 1B, 4B, 12B, 27B 
- 다국어 지원 : 140개 이상의 언어 지원, 특히 한국어 성능 우수
- 멀티모달 기능 : 텍스트뿐만 아니라 이미지, 비디오 분석 가능
- 확장된 컨텍스트 윈도우 : 최대 128K 토큰 처리 가능 
- 경량화 및 효율성 : 단일 GPU/TPU 및 저사양 기기에서도 높은 성능 발휘


## Use DJL with the SageMaker Python SDK
- SageMaker Python SDK를 사용하면 Deep Java Library를 이용하여 Amazon SageMaker에서 모델을 호스팅할 수 있습니다.
- Deep Java Library (DJL) Serving은 DJL이 제공하는 고성능 범용 독립형 모델 서빙 솔루션입니다. DJL Serving은 다양한 프레임워크로 학습된 모델을 로드하는 것을 지원합니다.
- SageMaker Python SDK를 사용하면 DeepSpeed와 HuggingFace Accelerate와 같은 백엔드를 활용하여 DJL Serving으로 대규모 모델을 호스팅할 수 있습니다.
- DJL Serving의 지원 버전에 대한 정보는 AWS 문서를 참조하십시오.
- 최신 지원 버전을 사용하는 것을 권장합니다. 왜냐하면 그곳에 우리의 개발 노력이 집중되어 있기 때문입니다.
- SageMaker Python SDK 사용에 대한 일반적인 정보는 SageMaker Python SDK 사용하기를 참조하십시오.
> REF: [BLOG] Deploy LLM with vLLM on SageMaker in only 13 lines of code


> DJLServing LMI 이미지 리스트 : https://github.com/aws/deep-learning-containers/blob/master/available_images.md

> HF 모델에 따라, Huggingface에서 사용 License를 요구하는 모델이 있습니다. Gemma3의 경우 라이선스를 요구 합니다. huggingface 참조

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

%pip install -qU sagemaker

Note: you may need to restart the kernel to use updated packages.


### 1. Depoly model on SageMaker

In [2]:
import boto3
import sagemaker
from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
role = get_execution_role()
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
sm_client = boto3.client("sagemaker", region_name=region)
sm_runtime_client = boto3.client("sagemaker-runtime")
sm_autoscaling_client = boto3.client("application-autoscaling")

### Setup Configuration

최신 버전 (DJL Serving 0.33)을 사용하는 경우, `sagemaker.image_uris.retrieve()`가 늦게 반영될 수도 있다. 

이 경우, container_uri 자체를 사용 하도록 합니다.  

ex) https://github.com/aws/deep-learning-containers/blob/master/available_images.md

> * 참조 : https://github.com/aws/deep-learning-containers/blob/master/available_images.md

[유의 사항]
- S3에 저장하여 사용하는 모델 데이터는 공식적인 가이드는 "압축 해제" 형태 이다. 
- 저장된 S3는 `model_data`에 입력하며, `model_data`는 prefix까지의 uri이다.

In [4]:
base_model = "Gemma-3"
# model_id = "google/gemma-3-12b-it"
model_id = "google/gemma-3-4b-it"
instance_type = "ml.g5.24xlarge"


bucket_uri = "s3://my-model-train/hf_models"
# model_data = f"{bucket_uri}/{model_id.split('/')[-1]}/model.tar.gz"
# S3 Prefix + 압축 해제(CompressionType=None)로 모델 마운트
model_data = {
    "S3DataSource": {
        "S3Uri": "s3://my-model-train/hf_models/gemma-3-4b-it-uncompressed/", 
        "S3DataType": "S3Prefix",
        "CompressionType": "None"
    }
}
print(model_data)

# 최신번전 : version 0.33 할려면, URI를 직접 사용하거나,  --> sagemaker upgrade 필요 : `uv pip install -U sagemaker`
# container_uri = sagemaker.image_uris.retrieve(
#     framework="djl-lmi", version="0.30.0", region=region
# )
# container_uri
container_uri = "763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.33.0-lmi15.0.0-cu128"
# 참조 : https://github.com/aws/deep-learning-containers/blob/master/available_images.md


{'S3DataSource': {'S3Uri': 's3://my-model-train/hf_models/gemma-3-4b-it-uncompressed/', 'S3DataType': 'S3Prefix', 'CompressionType': 'None'}}


In [5]:
container_startup_health_check_timeout = 900

endpoint_name = sagemaker.utils.name_from_base(base_model)

print (f'container_uri: {container_uri}')
print (f'container_startup_health_check_timeout: {container_startup_health_check_timeout}')
print (f'instance_type: {instance_type}')
print (f'endpoint_name: {endpoint_name}')

container_uri: 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.33.0-lmi15.0.0-cu128
container_startup_health_check_timeout: 900
instance_type: ml.g5.24xlarge
endpoint_name: Gemma-3-2025-08-13-09-26-48-970


### Creat model with env variables

- Target model: DeepSeek-Coder-V2-Light-Instruct
- Backend for attention computation in vLLM
- Available options:
    - "TORCH_SDPA": use torch.nn.MultiheadAttention
    - "FLASH_ATTN": use FlashAttention
    - "XFORMERS": use XFormers
    - "ROCM_FLASH": use ROCmFlashAttention
    - "FLASHINFER": use flashinfer


- '"OPTION_DISABLE_FLASH_ATTN": "false"' is for HF Accelerate with Seq-Scheduler
- It will be ignored when using vLLM beckend

> [DOC] DJL-Container and Model Configurations (info. about properties)

> [DOC] Backend Specific Configurations

In [7]:
HF_TOKEN = os.environ.get("HF_TOKEN","")

deploy_env = {
    # "HF_MODEL_ID": model_id,
    "OPTION_ROLLING_BATCH": "vllm",
    "OPTION_TENSOR_PARALLEL_DEGREE": "max",
    "OPTION_MAX_ROLLING_BATCH_SIZE": "8",
    # "OPTION_DTYPE":"fp16",
    "OPTION_DTYPE":"bf16",
    "OPTION_TRUST_REMOTE_CODE": "true",
    "OPTION_MODEL_IMPL": "transformers",
    # "OPTION_MAX_MODEL_LEN": "4096",
    # "VLLM_ATTENTION_BACKEND": "XFORMERS", # meta의 경우 XFORMERS, FlashAttention (default)
    #"OPTION_DISABLE_FLASH_ATTN": "false", ## HF Accelerate with Seq-Scheduler
    "HF_TOKEN": HF_TOKEN, # "<your token>"
}

In [8]:
model = sagemaker.Model(
    image_uri=container_uri,
    model_data=model_data,  # from S3 bucket - model.tar.gz
    role=role,
    env=deploy_env
)

### Deploy Model on Sagemaker

In [9]:
model.deploy(
    instance_type=instance_type,
    initial_instance_count=1,
    endpoint_name=endpoint_name,
    container_startup_health_check_timeout=container_startup_health_check_timeout,
    sagemaker_session=sagemaker_session
)

---------------------

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 model.deploy(                                                                                │
│   2 │   instance_type=instance_type,                                                             │
│   3 │   initial_instance_count=1,                                                                │
│   4 │   endpoint_name=endpoint_name,                                                             │
│                                                                                                  │
│ /home/sagemaker-user/user-default-efs/SKT/venv/lib/python3.11/site-packages/sagemaker/model.py:1 │
│ 814 in deploy                                                                                    │
│                                                                                                  │
│   1811 │   │   │   │   )                                                                         │
│   1812 │   │   │   │   self.sagemaker_session.update_endpoint(self.endpoint_name, endpoint_conf  │
│   1813 │   │   │   else:                                                                         │
│ ❱ 1814 │   │   │   │   self.sagemaker_session.endpoint_from_production_variants(                 │
│   1815 │   │   │   │   │   name=self.endpoint_name,                                              │
│   1816 │   │   │   │   │   production_variants=[production_variant],                             │
│   1817 │   │   │   │   │   tags=tags,                                                            │
│                                                                                                  │
│ /home/sagemaker-user/user-default-efs/SKT/venv/lib/python3.11/site-packages/sagemaker/session.py │
│ :6250 in endpoint_from_production_variants                                                       │
│                                                                                                  │
│   6247 │   │   logger.info("Creating endpoint-config with name %s", name)                        │
│   6248 │   │   self.sagemaker_client.create_endpoint_config(**config_options)                    │
│   6249 │   │                                                                                     │
│ ❱ 6250 │   │   return self.create_endpoint(                                                      │
│   6251 │   │   │   endpoint_name=name,                                                           │
│   6252 │   │   │   config_name=name,                                                             │
│   6253 │   │   │   tags=endpoint_tags,                                                           │
│                                                                                                  │
│ /home/sagemaker-user/user-default-efs/SKT/venv/lib/python3.11/site-packages/sagemaker/session.py │
│ :5084 in create_endpoint                                                                         │
│                                                                                                  │
│   5081 │   │   │   │   self.endpoint_arn = res["EndpointArn"]                                    │
│   5082 │   │   │                                                                                 │
│   5083 │   │   │   if wait:                                                                      │
│ ❱ 5084 │   │   │   │   self.wait_for_endpoint(endpoint_name, live_logging=live_logging)          │
│   5085 │   │   │   return endpoint_name                                                          │
│   5086 │   │   except Exception as e:                                                            │
│   5087 │   │   │   troubleshooting = (                     

### Invocation

In [ ]:
# endpoint_name = "Gemma-3-2025-08-12-13-47-53-107"
# endpoint_name = "Gemma-3-2025-08-13-02-12-55-046"

In [ ]:
predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),
)

In [ ]:
# 호출 예시
response = predictor.predict({
    # "inputs": "AI Agent에 대해 100단어 내외로 설명해 주세요.",
    "inputs": "tell me aboiut the AI Agent.",
    "parameters": {
        "max_new_tokens": 512,
        "stop": ["<|endoftext|>"]  # Stop sequences 지정 가능
    }
})

print(response)

{'generated_text': '<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

### Streaming Output

In [18]:
import json

def generate_payload(chat):
        
    # JSON 페이로드 생성
    body = {
        "messages": chat,
        "max_tokens": 512,
        "stream": True,
        "ignore_eos": False
    }
    
    # JSON을 문자열로 변환하고 bytes로 인코딩
    return json.dumps(body).encode('utf-8')

In [19]:
chat = [
    {"role": "system", "content": "너는 질의응답 챗봇입니다. 사용자의 질문의 의도를 파악하여 답변합니다. 답변은 한국어로 합니다"},
    {"role": "user", "content": "AWS AIML Specialist 솔루션즈 아키텍트 역할에 대해 설명해줘"},
]

In [20]:
%%time
# Invoke the endpoint
resp = sm_runtime_client.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name, 
    Body=generate_payload(chat),
    ContentType="application/json"
)
print("Generated response:")
print("-" * 40)

buffer = ""
string = "" 
for event in resp['Body']:
    if 'PayloadPart' in event:
        chunk = event['PayloadPart']['Bytes'].decode()
        buffer += chunk
        try:
            # Try to parse the buffer as JSON
            data = json.loads(buffer)
            if 'choices' in data:
                print(data['choices'][0]['delta']['content'], end='', flush=True)
                string += data['choices'][0]['delta']['content'] 
            buffer = ""  # Clear the buffer after successful parsing
        except json.JSONDecodeError:
            # If parsing fails, keep the buffer for the next iteration
            pass

print("\n" + "-" * 40)

CPU times: user 22.6 ms, sys: 2.16 ms, total: 24.7 ms
Wall time: 4.61 s


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 get_ipython().run_cell_magic('time', '', '# Invoke the endpoint\nresp = sm_runtime_clien     │
│   2                                                                                              │
│                                                                                                  │
│ /home/sagemaker-user/user-default-efs/SKT/venv/lib/python3.11/site-packages/IPython/core/interac │
│ tiveshell.py:2565 in run_cell_magic                                                              │
│                                                                                                  │
│   2562 │   │   │                                                                                 │
│   2563 │   │   │   with self.builtin_trap:                                                       │
│   2564 │   │   │   │   args = (magic_arg_s, cell)                                                │
│ ❱ 2565 │   │   │   │   result = fn(*args, **kwargs)                                              │
│   2566 │   │   │                                                                                 │
│   2567 │   │   │   # The code below prevents the output from being displayed                     │
│   2568 │   │   │   # when using magics with decorator @output_can_be_silenced                    │
│                                                                                                  │
│ /home/sagemaker-user/user-default-efs/SKT/venv/lib/python3.11/site-packages/IPython/core/magics/ │
│ execution.py:1470 in time                                                                        │
│                                                                                                  │
│   1467 │   │   │   print(f"Parser   : {_format_time(tp)}")                                       │
│   1468 │   │   if interrupt_occured:                                                             │
│   1469 │   │   │   if exit_on_interrupt and captured_exception:                                  │
│ ❱ 1470 │   │   │   │   raise captured_exception                                                  │
│   1471 │   │   │   return                                                                        │
│   1472 │   │   return out                                                                        │
│   1473                                                                                           │
│                                                                                                  │
│ /home/sagemaker-user/user-default-efs/SKT/venv/lib/python3.11/site-packages/IPython/core/magics/ │
│ execution.py:1434 in time                                                                        │
│                                                                                                  │
│   1431 │   │   else:                                                                             │
│   1432 │   │   │   st = clock2()                                                                 │
│   1433 │   │   │   try:                                                                          │
│ ❱ 1434 │   │   │   │   exec(code, glob, local_ns)                                                │
│   1435 │   │   │   │   out = None                                                                │
│   1436 │   │   │   │   # multi-line %%time case                                                  │
│   1437 │   │   │   │   if expr_val is not None:                                                  │
│ in <module>:2                                                                                    │
│                                                            

## AutoScaling

In [ ]:
import pprint
import random

In [ ]:
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)

# SageMaker expects resource id to be provided with the following structure
resource_id = f"endpoint/{endpoint_name}/variant/{resp['ProductionVariants'][0]['VariantName']}"

# Scaling configuration
scaling_config_response = sm_autoscaling_client.register_scalable_target(
    ServiceNamespace="sagemaker",
    ResourceId=resource_id,
    ScalableDimension="sagemaker:variant:DesiredInstanceCount", 
    MinCapacity=1,
    MaxCapacity=2

In [ ]:
# Create Scaling Policy
policy_name = f"scaling-policy-{endpoint_name}"
scaling_policy_response = sm_autoscaling_client.put_scaling_policy(
    PolicyName=policy_name,
    ServiceNamespace="sagemaker",
    ResourceId=resource_id,
    ScalableDimension="sagemaker:variant:DesiredInstanceCount",
    PolicyType="TargetTrackingScaling",
    TargetTrackingScalingPolicyConfiguration={
        "TargetValue": 5.0, # Target for avg invocations per minutes
        "PredefinedMetricSpecification": {
            "PredefinedMetricType": "SageMakerVariantInvocationsPerInstance",
        },
        "ScaleInCooldown": 600, # Duration in seconds until scale in
        "ScaleOutCooldown": 60 # Duration in seconds between scale out
    }
)

In [ ]:
response = sm_autoscaling_client.describe_scaling_policies(ServiceNamespace="sagemaker")

pp = pprint.PrettyPrinter(indent=4, depth=4)
for i in response["ScalingPolicies"]:
    pp.pprint(i["PolicyName"])
    print("")
    if("TargetTrackingScalingPolicyConfiguration" in i):
        pp.pprint(i["TargetTrackingScalingPolicyConfiguration"])

In [ ]:
# 다양한 코딩 태스크를 위한 프롬프트 리스트
prompts = [
    "write a quick sort algorithm in python.",
    "Write a Python function to implement a binary search algorithm.",
    "Create a JavaScript function to flatten a nested array.",
    "Implement a simple REST API using Flask in Python.",
    "Write a SQL query to find the top 5 customers by total purchase amount.",
    "Create a React component for a todo list with basic CRUD operations.",
    "Implement a depth-first search algorithm for a graph in C++.",
    "Write a bash script to find and delete files older than 30 days.",
    "Create a Python class to represent a deck of cards with shuffle and deal methods.",
    "Write a regular expression to validate email addresses.",
    "Implement a basic CI/CD pipeline using GitHub Actions."
]

def generate_payload():
    # 랜덤하게 프롬프트 선택
    prompt = random.choice(prompts)
    
    # JSON 페이로드 생성
    body = {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens": 400,
            # "return_full_text": False  # This does not work with Phi3
        },
        "stream": True,
    }
    
    # JSON을 문자열로 변환하고 bytes로 인코딩
    return json.dumps(body).encode('utf-8')

In [ ]:
%%time
import time

request_duration = 250
end_time = time.time() + request_duration
print(f"Endpoint will be tested for {request_duration} seconds")

while time.time() < end_time:
    payload = generate_payload()
    # Invoke the endpoint
    response = sm_runtime_client.invoke_endpoint_with_response_stream(
        EndpointName=endpoint_name, 
        # Body=json.dumps(body), 
        Body = payload,
        ContentType="application/json"
    )

In [ ]:
# Check the instance counts after the endpoint gets more load
response = sm_client.describe_endpoint(EndpointName=endpoint_name)
endpoint_status = response["EndpointStatus"]
request_duration = 250
end_time = time.time() + request_duration
print(f"Waiting for Instance count increase for a max of {request_duration} seconds. Please re run this cell in case the count does not change")
while time.time() < end_time:
    response = sm_client.describe_endpoint(EndpointName=endpoint_name)
    endpoint_status = response["EndpointStatus"]
    instance_count = response["ProductionVariants"][0]["CurrentInstanceCount"]
    print(f"Status: {endpoint_status}")
    print(f"Current Instance count: {instance_count}")
    if (endpoint_status=="InService") and (instance_count>1):
        break
    else:
        time.sleep(15)

### Clean Up

In [ ]:

# Delete model
sm_client.delete_model(ModelName=model_name)

# Delete endpoint configuration
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)

# Delete endpoint
sm_client.delete_endpoint(EndpointName=endpoint_name)